# Travel Chatbot

debug: [Langsmith](https://smith.langchain.com/o/9bf1c98b-4218-5f11-a048-6ee687b314f9/projects/p/9e4eea47-594f-4714-b02c-e511458be97f?timeModel=%7B%22duration%22%3A%227d%22%7D)

In [1]:
import os
import json
import pandas as pd
from typing import Literal
from dotenv import load_dotenv
from operator import itemgetter
from langchain_chroma import Chroma
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter, RecursiveJsonSplitter
from langchain.document_loaders import CSVLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from restaurant_chain import RestaurantChain

load_dotenv()

True

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

## Build chains for different use cases

In [4]:
def chain_creator(llm, prompt_template):
    return prompt_template | llm | StrOutputParser()

def create_prompt_template(system_message: str) -> ChatPromptTemplate:
    return ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            ("human", "{query}"),
        ]
    )

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Define system messages for each service
service_prompts = {
    "tourist_attraction": "You are an expert in finding tourist attractions.",
    "itinerary_planning": "You are an expert in travel itinerary planning.",
    "exploring_travel_ideas": "You are an expert in recommending travel ideas.",
    "others": "You are a friendly and helpful chatbot."
}

# Create chains for each service (service without Rag)
service_chains = {service: chain_creator(llm=llm, prompt_template=create_prompt_template(service_prompts[service])) for service in service_prompts.keys()}

# Create chains for each service (service with Rag)
restaurantChain = RestaurantChain()
restaurant_full_chains = restaurantChain.get_full_chains()
service_chains['restaurant_recommendations'] = restaurant_full_chains

## Route an input query to a specific chain 

In [12]:
# Define the routing system
class RouteQuery(TypedDict):
    """Route query to the appropriate service_type."""
    service_type: Literal[
        "tourist_attraction", 
        "itinerary_planning", 
        "restaurant_recommendations", 
        "exploring_travel_ideas",
        "others"
    ]

route_system = "Route the user's query to either 'tourist_attraction', 'itinerary_planning', 'restaurant_recommendations', 'exploring_travel_ideas', or 'others' if it doesn't fit into the previous categories."

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{query}"),
    ]
)

# Routing chain based on the query
route_chain = route_prompt | llm.with_structured_output(RouteQuery) | itemgetter("service_type")

# Final chain that routes to the appropriate LLM chain based on user input
def get_chain_for_service(input):
    service_type = input['service_type']
    query = input['query']
    print("service_type: ", service_type)
    print("query: ", query)
    if service_type in service_chains.keys():
        return service_chains.get(service_type).invoke({"query": query})
    else:
        raise ValueError(f"Unknown service type: {service_type}")
    
# Combine all the steps into the final chain
chatbot_chain = {
    "service_type": route_chain,
    "query": lambda x: x['query']
} | RunnableLambda(get_chain_for_service)

## Start

In [13]:
if __name__ == "__main__":

    inout_query = {
        "query": "What's the weather tomorrow in Victoria BC ?"
    }    
    response = chatbot_chain.invoke(inout_query)
    print(f"Response: {response}")

service_type:  others
query:  What's the weather tomorrow in Victoria BC ?
Response: I don't have real-time information on weather. However, you can check the weather forecast for Victoria, BC tomorrow by visiting a weather website like Weather.com or by using a weather app on your smartphone.
